In [ ]:
# B.Tech Project, Indian Insititute of Technology, Delhi
# Authors - Harshita (2019CS190272), Om Krishna (2019CS10272)
# Input : Statewise lithology shapefiles from Bhukosh

import requests
import rasterio as rio
from rasterio.enums import Resampling
from rasterio.plot import show
import numpy as np
from matplotlib import pyplot
from PIL import Image
import osgeo
from osgeo import gdal, ogr
import geopandas as gpd
import os
from tqdm import tqdm

In [ ]:
import csv

# CSV file with state Lat1 Long1 Lat2 Long2, and state codes
# https://docs.google.com/spreadsheets/d/1Te7I-JH1sTqC_e6MOMegvVFpeUneAt7c7NYqn3A787Q/edit?usp=sharing
# ['AndamanNicobar', '92.208', '6.758', '93.948', '13.676', 'AN']
# Tripura, Mizoram Data Not Available, J&K Incomplete 

with open('lintemp/bhuvan_extent.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

data = data[1:]
data 

In [ ]:
# folder names for output files

folder_bhukosh_litho = 'bhukosh_litho/'
out_folder = folder_bhukosh_litho + 'all_scores/'
temp_folder = folder_bhukosh_litho + 'all_temp/'

In [ ]:
# function to rasterize from vector data to GeoTiff

def rasterize(s, df_header, soil, burnval): # -ts, -te can be arguments (ts computed from te or let default - ?)
    v_soil = temp_folder + s[5] + '_' + soil + '.shp'
    # create soil type shapefile
    df[df[df_header].str.contains(soil, case=False)==True].to_file(v_soil) 
    
    while not os.path.exists(v_soil):
        time.sleep(1)
    
    # rasterize created shapefile
    r_soil = temp_folder + s[5] + '_' + soil + '_r.tif'
    
    options_list = [ '-of GTiff', '-burn ' + burnval, '-ts 2000 2000', 
                     '-te ' + str(s[1]) + ' ' + str(s[2]) + ' ' + str(s[3]) + ' ' + str(s[4]), # L1 L4 L3 L2 
                     '-a_nodata -1', '-ot Int32', '-init 0']

    options_string = " ".join(options_list)
    ds = gdal.Rasterize(r_soil, v_soil, options=options_string)
    ds = None

In [ ]:
for s in data:
    litho = folder_bhukosh_litho + s[5] + '/Lithology.shp' 
    df = gpd.read_file(litho)
    
    # 5 major geology types that exist, can be extended for other types
    # Value is a measure of infiltration factor(acc to GEC norms), smaller value = more water seeps in = better recharge potential
    rasterize(s, 'GROUP_NAME', 'ALLUVIUM', '1')
    rasterize(s, 'LITHOLOGIC', 'GRANITE', '2')
    rasterize(s, 'LITHOLOGIC', 'SCHIST', '2')
    rasterize(s, 'LITHOLOGIC', 'GNEISS', '2')
    rasterize(s, 'LITHOLOGIC', 'PHYLLITE', '2')
    rasterize(s, 'LITHOLOGIC', 'QUARTZ', '3')
    
    rastersloc = temp_folder + s[5] + '_'
    while not os.path.exists(rastersloc + 'ALLUVIUM_r.tif'):
        time.sleep(1)
    while not os.path.exists(rastersloc + 'GRANITE_r.tif'):
        time.sleep(1)
    while not os.path.exists(rastersloc + 'SCHIST_r.tif'):
        time.sleep(1)
    while not os.path.exists(rastersloc + 'GNEISS_r.tif'):
        time.sleep(1)
    while not os.path.exists(rastersloc + 'QUARTZ_r.tif'):
        time.sleep(1)
    while not os.path.exists(rastersloc + 'PHYLLITE_r.tif'):
        time.sleep(1)
        
    alu = rio.open(rastersloc + 'ALLUVIUM_r.tif').read(1).astype(float)
    gra = rio.open(rastersloc + 'GRANITE_r.tif').read(1).astype(float)
    sst = rio.open(rastersloc + 'SCHIST_r.tif').read(1).astype(float)
    gns = rio.open(rastersloc + 'GNEISS_r.tif').read(1).astype(float)
    qtz = rio.open(rastersloc + 'QUARTZ_r.tif').read(1).astype(float)
    phy = rio.open(rastersloc + 'PHYLLITE_r.tif').read(1).astype(float)

    m, n = phy.shape # all 2000 x 2000 : ts
    print(m,n)

    # new raster
    litho = np.zeros((m,n))
    lithotif = temp_folder + s[5] + '_litho.tif'

    # merging individual rasters to one output raster
    for i in tqdm(range(0,m)):
        for j in range (0,n):

            # infiltration factor is limited by the worst infiltration (largest value hence max)
            litho[i][j] = max(alu[i][j], gra[i][j], sst[i][j], gns[i][j], qtz[i][j], phy[i][j]) 
            

    out_tiff = Image.fromarray(litho)
    out_tiff.save(lithotif)


    # georeff jamui
    georeffed = out_folder + s[5] + '_geo_litho.tif'
    while not os.path.exists(lithotif):
        time.sleep(1)
    ds = gdal.Open(lithotif)

    options_list = [
        '-of GTiff',
        '-a_srs EPSG:4326',
        '-a_ullr ' + str(s[1]) + ' ' + str(s[4]) + ' ' + str(s[3]) + ' ' + str(s[2]) # L1 L4 L3 L2
    ] 

    options_string = " ".join(options_list)

    ds = gdal.Translate(georeffed,
                        lithotif,
                        options=options_string)

    ds = None
    
    print("Lithology mask for " + s[0] + " generated successfully.")